
> This Notebook aims to share the idea of using Tensorflow-lite for inference. 
> The NN model and the way I'm splitting/training the data is very naive and was done just for code completion.

In [ ]:
import numpy as np
import cudf
import datatable as dt

import tensorflow as tf
import janestreet

from tensorflow.keras.layers import Input, BatchNormalization, Dense, Dropout, Activation
from tensorflow.keras.losses import binary_crossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.activations import swish

from tqdm.notebook import tqdm

In [ ]:
TRAIN_PATH = '../input/jane-street-market-prediction/train.csv'
BATCH = 4096
EPOCHS = 100

### Loading the data

In [ ]:
%%time

data = dt.fread(TRAIN_PATH)
data = data.sort('ts_id')

# Converting to pandas DataFrame
df = data.to_pandas()

# Remove rows where weight = 0 
df = df[df.weight != 0]

# Fill NaN values both ways
df = df.fillna(method='ffill').fillna(method='bfill')

# Create the target data
df['action'] = (df['resp'] > 0).astype(np.int32)

In [ ]:
features = [ f'feature_{i}' for i in range(130)]

### NN Model

This is a very simple NN model just for tests.

In [ ]:
def create_model():
    
    model = tf.keras.Sequential([ 
        
        Input(len(features)),
        BatchNormalization(),
        Dropout(0.1),
        
        Dense(128),
        BatchNormalization(),
        Activation(swish),
        Dropout(0.1),
        
        Dense(64),
        BatchNormalization(),
        Activation(swish),
        Dropout(0.1),

        Dense(1,activation='sigmoid')
    ])
    
    model.compile(loss=binary_crossentropy, optimizer=Adam(0.01), metrics=AUC(name = 'AUC'))
    
    return model

### Splitting tha data

For test purpose I'll validate on last day. There are plenty of notebooks describing the proper way to split time series data.

In [ ]:
train = df[df['date'] < 499]
val = df[df['date'] == 499]

X_tr = train[features].values
y_tr = train['action'].values

X_val = val[features].values
y_val = val['action'].values


In [ ]:
model = create_model()
model.fit(X_tr, y_tr,
          validation_data=(X_val, y_val),
          epochs=EPOCHS, 
          batch_size=BATCH,
          verbose=1)

## Converting the model

Let's convert our model to be a **Tensorflow-lite model**

I got the code from: [using-tensorflow-lite-to-speed-up-predictions](https://medium.com/@micwurm/using-tensorflow-lite-to-speed-up-predictions-a3954886eb98)

In [ ]:
# From https://medium.com/@micwurm/using-tensorflow-lite-to-speed-up-predictions-a3954886eb98

class LiteModel:
    
    @classmethod
    def from_file(cls, model_path):
        return LiteModel(tf.lite.Interpreter(model_path=model_path))
    
    @classmethod
    def from_keras_model(cls, kmodel):
        converter = tf.lite.TFLiteConverter.from_keras_model(kmodel)
        tflite_model = converter.convert()
        return LiteModel(tf.lite.Interpreter(model_content=tflite_model))
    
    def __init__(self, interpreter):
        self.interpreter = interpreter
        self.interpreter.allocate_tensors()
        input_det = self.interpreter.get_input_details()[0]
        output_det = self.interpreter.get_output_details()[0]
        self.input_index = input_det["index"]
        self.output_index = output_det["index"]
        self.input_shape = input_det["shape"]
        self.output_shape = output_det["shape"]
        self.input_dtype = input_det["dtype"]
        self.output_dtype = output_det["dtype"]
        
    def predict(self, inp):
        inp = inp.astype(self.input_dtype)
        count = inp.shape[0]
        out = np.zeros((count, self.output_shape[1]), dtype=self.output_dtype)
        for i in range(count):
            self.interpreter.set_tensor(self.input_index, inp[i:i+1])
            self.interpreter.invoke()
            out[i] = self.interpreter.get_tensor(self.output_index)[0]
        return out
    
    def predict_single(self, inp):
        """ Like predict(), but only for a single record. The input data can be a Python list. """
        inp = np.array([inp], dtype=self.input_dtype)
        self.interpreter.set_tensor(self.input_index, inp)
        self.interpreter.invoke()
        out = self.interpreter.get_tensor(self.output_index)
        return out[0]

In [ ]:
# Model conversion
tflite_model = LiteModel.from_keras_model(model)

In [ ]:
env = janestreet.make_env()
env_iter = env.iter_test()

In [ ]:
opt_th = 0.5

for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        test_df = test_df.fillna(method='ffill').fillna(method='bfill')
        x_tt = test_df.loc[:, features].values
        pred = tflite_model.predict(x_tt)
        pred_df.action = np.where(pred >= opt_th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)